# 1. Create schema in HMS

In [2]:
%%sql

-- Check metastore status
--show schemas from hms;
--show tables from hms.nyc100;
--drop table hms.nyc_taxis_db.test;
--drop schema hms.nyc;

CREATE DATABASE nyc_taxis_db LOCATION 's3a://datalake/warehouse/nyc_taxis_db';

++
||
++
++

# 2. Spark example with Python API

In [1]:
initial_data_df = spark.read.parquet("s3a://datalake/files/sales_summary")
initial_data_df.show()

24/03/26 21:01:17 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+-------------------+----------------------+--------------------+------------------+---------------------+----------------+-------------+----------------+-----------+--------+-------------------+
|ProductCategoryName|ProductSubcategoryName|         ProductName|      CustomerName|SalesTerritoryCountry|SalesOrderNumber|OrderQuantity|TotalProductCost|SalesAmount|  TaxAmt|          OrderDate|
+-------------------+----------------------+--------------------+------------------+---------------------+----------------+-------------+----------------+-----------+--------+-------------------+
|              Bikes|            Road Bikes|    Road-150 Red, 62|       Cole Watson|               Canada|         SO43697|            1|       2171.2942|    3578.27|286.2616|2010-12-29 00:00:00|
|              Bikes|        Mountain Bikes|Mountain-100 Silv...|  Rachael Martinez|               France|         SO43698|            1|       1912.1544|    3399.99|271.9992|2010-12-29 00:00:00|
|              Bikes

## 2.1 Create Iceberg table in HMS catalog

In [2]:
TAXIS_TABLE="hms.nyc_taxis_db.trips"

In [7]:
initial_data_df.writeTo(TAXIS_TABLE).create()

## 2.3 Read Hive table

In [8]:
spark.read.table(TAXIS_TABLE).show()

+-------------------+----------------------+--------------------+------------------+---------------------+----------------+-------------+----------------+-----------+--------+-------------------+
|ProductCategoryName|ProductSubcategoryName|         ProductName|      CustomerName|SalesTerritoryCountry|SalesOrderNumber|OrderQuantity|TotalProductCost|SalesAmount|  TaxAmt|          OrderDate|
+-------------------+----------------------+--------------------+------------------+---------------------+----------------+-------------+----------------+-----------+--------+-------------------+
|              Bikes|            Road Bikes|    Road-150 Red, 62|       Cole Watson|               Canada|         SO43697|            1|       2171.2942|    3578.27|286.2616|2010-12-29 00:00:00|
|              Bikes|        Mountain Bikes|Mountain-100 Silv...|  Rachael Martinez|               France|         SO43698|            1|       1912.1544|    3399.99|271.9992|2010-12-29 00:00:00|
|              Bikes

# 3. Spark SQL example

In [34]:
%%sql
CREATE TABLE hms.nyc100.taxis_new
(
  vendor_id bigint,
  trip_id bigint,
  trip_distance float,
  fare_amount double,
  store_and_fwd_flag string
)
PARTITIONED BY (vendor_id);

++
||
++
++

In [2]:
%%sql
INSERT INTO hms.nyc100.taxis_new
VALUES (1, 1000371, 1.8, 15.32, 'N'), (2, 1000372, 2.5, 22.15, 'N'), (2, 1000373, 0.9, 9.01, 'N'), (1, 1000374, 8.4, 42.13, 'Y');

++
||
++
++

In [3]:
%%sql
SELECT * FROM hms.nyc100.taxis_new;

vendor_id,trip_id,trip_distance,fare_amount,store_and_fwd_flag
1,1000371,1.7999999523162842,15.32,N
1,1000374,8.399999618530273,42.13,Y
2,1000372,2.5,22.15,N
2,1000373,0.8999999761581421,9.01,N
1,1000371,1.7999999523162842,15.32,N
1,1000374,8.399999618530273,42.13,Y
2,1000372,2.5,22.15,N
2,1000373,0.8999999761581421,9.01,N
